In [1]:
import sys
import os
import json
import tensorflow as tf
import tqdm

from ltv_utils import *
pd.set_option('display.float_format', '{:.4f}'.format)  # 保留10位小数，可调整
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import psutil
dense_vector_length = 121

train_file_dir = None

df = load_obj("data/test_data.obj")
print(df)

       all_install_order_7d_diff  b2_sale_amt_7d  b2_sale_amt_bias channel1  \
0                             14             237             50000        9   
1                       -7593043               0             50000       腾讯   
2                       -7580183               0             50000       腾讯   
3                       -7589881               0             50000        8   
4                       -7597435               0             50000  今日头条2.0   
...                          ...             ...               ...      ...   
99995                   -7579734               0             50000  今日头条2.0   
99996                   -7574285               0             50000       25   
99997                   -7593653               0             50000  今日头条2.0   
99998                   -7584129               0             50000       25   
99999                   -7573824               0             50000       24   

      creative_classify1                           

In [4]:
df

,all_install_order_7d_diff,b2_sale_amt_7d,b2_sale_amt_bias,channel1,creative_classify1,dense_vector,deviceid,dim_os_name1,install_date,install_order_diff,install_time,is_a1x_a33,platform_label,total_pay_amount1
0,14,237,50000,9,None,"[2.0, 1.0, 1.0, 1.0, 0.0, 4.0, 1.0, 1.0, 0.0, ...",cca45008-8bfd-3a39-8877-3a700bd8c323,android,2025-05-27,14,2025-05-27 16:09:09,1,b'a1-0',237.1000
1,-7593043,0,50000,腾讯,None,"[3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",59b144cc-5274-372b-9168-e0935b13473b,harmonyos,2025-06-07,-7593043,2025-06-07 22:43:47,1,b'a1-4',0.0000
2,-7580183,0,50000,腾讯,None,"[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",d837aee3-677b-3971-8d2c-22e88162ca85,android,2025-05-30,-7580183,2025-05-30 00:23:06,1,b'a1-0',0.0000
3,-7589881,0,50000,8,None,"[3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, ...",597d813b-d16b-4cbb-bd44-404093a9ec45,ios,2025-06-05,-7589881,2025-06-05 18:01:41,1,b'a1-0',0.0000
4,-7597435,0,50000,今日头条2.0,保养,"[4.0, 1.0, 1.0, 0.0, 1.0, 3.0, 0.0, 1.0, 0.0, ...",8df253d9-69f4-4ea4-be0e-4ea42e0f4d25,ios,2025-06-10,-7597435,2025-06-10 23:55:48,1,b'a1-1',0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-7579734,0,50000,今日头条2.0,保养,"[3.0, 1.0, 1.0, 0.0, 1.0, 3.0, 1.0, 3.0, 0.0, ...",8e1abf29-abe3-4bd7-9dba-1fd03adcea99,ios,2025-05-29,-7579734,2025-05-29 16:54:17,1,b'a1-5',0.0000
99996,-7574285,0,50000,25,None,"[3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, ...",5ee311d0-6bd6-3c24-ae60-0979a28b961d,harmonyos,2025-05-25,-7574285,2025-05-25 22:05:43,1,b'a1-0',0.0000
99997,-7593653,0,50000,今日头条2.0,保养,"[5.0, 1.0, 1.0, 1.0, 1.0, 4.0, 0.0, 3.0, 0.0, ...",1c03a9ee-5dae-33f5-9d13-b97a20988b1c,android,2025-06-08,-7593653,2025-06-08 08:53:38,0,b'a3-2',0.0000
99998,-7584129,0,50000,25,None,"[3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",fbf78892-8e40-3f40-bff6-215781d53c78,harmonyos,2025-06-01,-7584129,2025-06-01 18:09:41,1,b'a3-3',0.0000
